# SQL II: Grouping and Joining

_November 2, 2020_

Agenda:
- Practice with Grouping in SQLite
- Practice with different types of Joins in SQLite

<img src = "https://media.giphy.com/media/3oKIPnAiaMCws8nOsE/giphy.gif">

In [1]:
import sqlite3
import pandas as pd
import seaborn as sns
import numpy as np

## Part I. Grouping
Just like pandas, Sql support different types of grouping statements for performing aggregate functions and allow us to calculate various statistics for data.

Syntax:
`SELECT (*) FROM tables GROUP BY column`

`dr.groupby(['columns']).columns(*).operations()`

In [13]:
# instantiate some data
np.random.seed(1234)

auto = pd.read_csv('auto-mpg.csv')
tips = sns.load_dataset('tips')
small_quiz = pd.DataFrame({"students":["Billy","Ijeoma","Kelvin"],
                          "projects":[1,2,1],
                          "grades":np.random.randint(80,100,3)})
small_grades = pd.DataFrame({"students":["Muhammad","Joe","Kelvin"],
                            "quiz_score":np.random.randint(0,10,3)})

In [14]:
#tips.to_csv('tips.csv')

# tips = pd.read_csv('tips.csv')

In [15]:
conn = sqlite3.connect('aggregate.db')
c = conn.cursor()

In [18]:
# insert these datasets into our db
#NO NEED TO RUN AGAIN

# auto.to_sql('auto', con=conn)

# tips.to_sql('tips', con=conn)
# small_grades.to_sql('small_grade', con=conn)
# small_quiz.to_sql('small_quiz',con=conn)

In [19]:
# for the tips df/table, get the total number of man and woman

tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [27]:
# in pandas

tips.groupby(['sex']).size()

sex
Male      157
Female     87
dtype: int64

In [35]:
#alternatives
grouped_by_sex = tips.groupby(['sex']).sex.value_counts()
grouped_by_sex.values

array([157,  87])

In [34]:
grouped_by_sex.values[1

87

Syntax: `SELECT ___ FROM ___ GROUP BY ___` 

In [36]:
# in sql 
query = """SELECT COUNT(sex), sex FROM tips GROUP BY sex"""
c.execute(query).fetchall()

[(87, 'Female'), (157, 'Male')]

In [37]:
# using pandas, select the average amount tipped for time
pd_query = tips.groupby(['time'])['tip'].mean()
pd_query

time
Lunch     2.728088
Dinner    3.102670
Name: tip, dtype: float64

In [41]:
# use sql to recreate this query
query = """SELECT time, ROUND(AVG(tip),3) FROM tips GROUP BY time"""
c.execute(query).fetchall()

[('Dinner', 3.103), ('Lunch', 2.728)]

We can also use `GROUP BY` with certain conditions, in conjunction with filtering and ordering.

You can use :

`SELECT ___ FROM ___ WHERE ___ GROUP BY ___ `

`SELECT ___ FROM ___ GROUP BY ___ HAVING ____ `

In [55]:
# use pandas, get the average total bills for a party larger than 2 for every size of party

# using pandas

tips[tips['size'] > 2].groupby(['size'])['total_bill'].mean()

size
3    23.277632
4    28.613514
5    30.068000
6    34.830000
Name: total_bill, dtype: float64

In [45]:
# use sql to recreate this query
query = """SELECT AVG([total_bill]), size FROM tips WHERE size > 2 GROUP BY size"""
c.execute(query).fetchall()

[(23.27763157894737, 3), (28.61351351351351, 4), (30.068, 5), (34.83, 6)]

In [65]:
# your turn - use pandas, get the maximum value of total bill for female non smoker

# using pandas?
tips[(tips['sex']=='Female') & (tips['smoker'] == 'No')]['total_bill'].max()


35.83

In [ ]:
# Hint - you can use this with out without group by


In [76]:
# use sql to recreate this query
query = """SELECT MAX([total_bill])
            FROM tips 
            WHERE (sex = "Female") AND (smoker = 'No') """
c.execute(query).fetchall()*10

[(35.83,),
 (35.83,),
 (35.83,),
 (35.83,),
 (35.83,),
 (35.83,),
 (35.83,),
 (35.83,),
 (35.83,),
 (35.83,)]

In [62]:
# your turn - use pandas, get the average tip values for different size groups, and show top 2
# most amount of average tips

tips.groupby(['size'])['tip'].mean().sort_values(ascending = False).head()[:2]


size
6    5.225000
4    4.135405
Name: tip, dtype: float64

In [59]:
# use sql 
query = """SELECT AVG(tip), size FROM tips GROUP BY size ORDER BY AVG(tip) DESC LIMIT 2"""
c.execute(query).fetchall()

[(5.225, 6), (4.135405405405407, 4)]

## Part II. Join

Based on this diagram, how would you explain the different types of joins?

<img src = 'sql-joins-better.png' width = 400>

In [79]:
print(small_grades)
print(small_quiz)

   students  quiz_score
0  Muhammad           5
1       Joe           4
2    Kelvin           8
  students  projects  grades
0    Billy         1      95
1   Ijeoma         2      99
2   Kelvin         1      86


In [81]:
# left join - join small quiz with small grades, on student name
small_grades.merge(small_quiz, on = "students", how  ='left')

,students,quiz_score,projects,grades
0,Muhammad,5,NaN,NaN
1,Joe,4,NaN,NaN
2,Kelvin,8,1.0,86.0


In [82]:
# sql 
query = """SELECT * FROM small_grade LEFT JOIN small_quiz on small_grade.students = small_quiz.students"""
c.execute(query).fetchall()
col = [i[0] for i in c.description]

In [83]:
# inner join and get the student's grades
small_grades.merge(small_quiz, on = 'students')

,students,quiz_score,projects,grades
0,Kelvin,8,1,86


In [ ]:
query = """SELECT * FROM small_quiz JOIN small_grade ON small_grade.students = small_quiz.students"""
c.execute(query).fetchall()

In [ ]:
# your turn - use right join and get all entries

# pandas

In [ ]:
# sql 

### Level up 
1. Subqueries (including anti-joins)
2. Get median (using OFFSET)

In [ ]:
# anti-join :get all the entries that are in the left table but not in the right table 
query = """SELECT left_table.id FROM `left table` WHERE left_table.id not in (SELECT id FROM right_table) """

In [ ]:
# get median - use offset 
query = """SELECT col FROM table ORDER BY col LIMIT 1 OFFSET (SELECT COUNT(*) FROM MyTable) / 2"""